In [1]:
#load library
import numpy as np
import pandas as pd
#load data
df = pd.read_csv("IMDB-Movie-Data.csv")

In [2]:
df.head()#print 5 row to see structure

,Rank,Title,Genre,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore
0,55,The Dark Knight,Action|Crime|Drama,Christopher Nolan,Christian Bale| Heath Ledger| Aaron Eckhart|Mi...,2008,152,9.0,1791916,533.32,82.0
1,81,Inception,Action|Adventure|Sci-Fi,Christopher Nolan,Leonardo DiCaprio| Joseph Gordon-Levitt| Ellen...,2010,148,8.8,1583625,292.57,74.0
2,118,Dangal,Action|Biography|Drama,Nitesh Tiwari,Aamir Khan| Sakshi Tanwar| Fatima Sana Shaikh|...,2016,161,8.8,48969,11.15,NaN
3,37,Interstellar,Adventure|Drama|Sci-Fi,Christopher Nolan,Matthew McConaughey| Anne Hathaway| Jessica Ch...,2014,169,8.6,1047747,187.99,74.0
4,97,Kimi no na wa,Animation|Drama|Fantasy,Makoto Shinkai,Ryunosuke Kamiki| Mone Kamishiraishi| Ryo Nari...,2016,106,8.6,34110,4.68,79.0


### Question1
Top‐3 movies with the highest ratings in 2016? 

In [3]:
#2016 as condition
df_2016 = df[df['Year']==2016]
#sort by descending and get the first three row
df_2016_top3 = df_2016.sort_values(['Rating'], ascending=False).head(3)
df_2016_top3['Title']

2             Dangal
4      Kimi no na wa
15    Koe no katachi
Name: Title, dtype: object

### Question2
The actor generating the highest average revenue? 

In [4]:
# split actors and preprocess
df_actors = df['Actors'].str.split('|',expand=True).stack()
df_actors = df_actors.reset_index(level=1,drop=True).rename('actor')
df_actors = df_actors.str.replace(' ','')
df_actor = df.drop(['Actors'],axis=1).join(df_actors)
#get revenue and actor
df_actor_rev = df_actor[['Revenue (Millions)','actor']]
df_actor_rev.groupby('actor').agg('mean').idxmax()

Revenue (Millions)    DaisyRidley
dtype: object

In [5]:
# df_actor

### Question3
The average rating of Emma Watson’s movies? 

In [6]:
#use EmmaWatson as condition
df_emma = df_actor[df_actor['actor']=='EmmaWatson']
# calculate average rate
df_emma['Rating'].agg('average')

7.175000000000001

### Question4
Top‐3 directors who collaborate with the most actors? 

In [7]:
#get director, actor columns
dir_actor = df_actor[['Director','actor']]
director = dir_actor['Director'].unique()
dir_count = pd.DataFrame(columns = ['unique actor count'])
for i in director: 
    a = dir_actor.groupby('Director').get_group(i)['actor'].unique()
    dir_count.loc[i] = len(a)
dir_count.sort_values(by=['unique actor count'],ascending=False).head(3).index

Index(['Ridley Scott', 'M. Night Shyamalan', 'Danny Boyle'], dtype='object')

### Question5
Top‐2 actors playing in the most genres of movies?

In [8]:
#split genre str to column
df_genres = df_actor['Genre'].str.split('|',expand=True).stack()
df_genres = df_genres.reset_index(level=1,drop=True).rename('genre')
df_genres = df_genres.str.replace(' ','')
df_genre = df_actor.drop(['Genre'],axis=1).join(df_genres)
genre_actor = df_genre[['actor','genre']]
#count unique genre
actor = genre_actor['actor'].unique()
genre_count = pd.DataFrame(columns = ['unique genre count'])
for i in actor: 
    a = genre_actor.groupby('actor').get_group(i)['genre'].unique()
    genre_count.loc[i] = len(a)
genre_count.sort_values(by=['unique genre count'],ascending=False).head(2).index

Index(['BradPitt', 'HughJackman'], dtype='object')

### Question6
Top‐3 actors whose movies lead to the largest maximum gap of years? 

In [9]:
actor_max_min = pd.DataFrame(columns=['year'])
#go through actor list get max and min year calculate gap
for i in actor: 
    max_year = df_actor.groupby('actor').get_group(i)['Year'].max()
    min_year = df_actor.groupby('actor').get_group(i)['Year'].min()
    actor_max_min.loc[i] = max_year-min_year
actor_max_min.sort_values(['year'], ascending=False).head(3)

,year
ChristianBale,10
KateBosworth,10
RachelWeisz,10


### Question7
Find all actors who collaborate with Johnny Depp in direct and indirect ways

In [189]:
ls_JD_movie = []
ls_JD_actor = []
#unique JohnnyDepp movie lists
for j in df_actor[df_actor['actor']=='JohnnyDepp']['Title']:
    ls_JD_movie.append(j)
ls_JD_movie = list(set(ls_JD_movie))
#unique JohnnyDepp related actor lists
for i in ls_JD_movie:
    for j in df_actor[df_actor['Title']==i]['actor']:
        ls_JD_actor.append(j)
pre_temp = len(list(set(ls_JD_actor)))
#get second lists
for i in ls_JD_actor:
    for j in df_actor[df_actor['actor']==i]['Title']:
        if j not in ls_JD_movie:
            ls_JD_movie.append(j)
for i in ls_JD_movie:
    for j in df_actor[df_actor['Title']==i]['actor']:
        if j not in ls_JD_actor:
            ls_JD_actor.append(j)
temp = len(ls_JD_actor)
#go through actor list until find all
while(temp!= pre_temp):
    pre_temp = temp
    for i in ls_JD_actor:
        for j in df_actor[df_actor['actor']==i]['Title']:
            if j not in ls_JD_movie:
                ls_JD_movie.append(j)
    for i in ls_JD_movie:
        for j in df_actor[df_actor['Title']==i]['actor']:
            if j not in ls_JD_actor:
                ls_JD_actor.append(j)
    temp = len(ls_JD_actor)
ls_JD_actor

['JohnnyDepp',
 'OrlandoBloom',
 'KeiraKnightley',
 'JackDavenport',
 'JohnnyDepp',
 'HelenaBonhamCarter',
 'AlanRickman',
 'TimothySpall',
 'MiaWasikowska',
 'JohnnyDepp',
 'HelenaBonhamCarter',
 'AnneHathaway',
 'ChristianBale',
 'JohnnyDepp',
 'ChristianStolte',
 'JasonClarke',
 'JohnnyDepp',
 'ArmieHammer',
 'WilliamFichtner',
 'TomWilkinson',
 'JohnnyDepp',
 'BenedictCumberbatch',
 'DakotaJohnson',
 'JoelEdgerton',
 'JohnnyDepp',
 'MichellePfeiffer',
 'EvaGreen',
 'HelenaBonhamCarter',
 'JohnnyDepp',
 'RebeccaHall',
 'MorganFreeman',
 'CillianMurphy',
 'JohnnyDepp',
 'PenelopeCruz',
 'IanMcShane',
 'GeoffreyRush',
 'MiaWasikowska',
 'JohnnyDepp',
 'HelenaBonhamCarter',
 'AnneHathaway',
 'JohnnyDepp',
 'OrlandoBloom',
 'KeiraKnightley',
 'GeoffreyRush',
 'Lily-RoseDepp',
 'HarleyQuinnSmith',
 'JohnnyDepp',
 'AdamBrody',
 'MatthewGoode',
 'AllenLeech',
 'JamesMcAvoy',
 'BrendaBlethyn',
 'SaoirseRonan',
 'MarkRuffalo',
 'AdamLevine',
 'HaileeSteinfeld',
 'JudeLaw',
 'AaronTaylor-John